# Load Train Set

features (3681) | labels (1)

In [12]:
# import pandas as pd
import cudf
import cupy
import pandas as pd
import numpy as np

# df = pd.read_feather("dts/np_dataset_train.ftr")
df = pd.read_csv("dts/np_dataset_train.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 3682 entries, img_0 to Label
dtypes: bool(1), float64(3681)
memory usage: 1.1 GB


## train/test Split

In [13]:
from sklearn.model_selection import train_test_split

# rs = 37

# train, test = train_test_split(df, test_size=0.2, random_state=rs)
# train.info()
# test.info()

## Classifier definition

In [14]:
from random import seed
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier as tree_classifier
from scipy import stats
import cuml

# from cuml import tree_classifier


class random_forest():
    def __init__(self, number_of_trees=1, number_of_iteractions=1, max_depth=1, random_state=1):
        self.iteractions = number_of_iteractions
        self.number_of_trees = number_of_trees
        self.max_depth = max_depth
        self.forest = []
        self.random_state = random_state

    def bootstrap(self, x_samples):
        """ Create subsets of the input, and returns the indexes 
            from the original sample."""
        index = []

        sample_length = int(np.floor(len(x_samples)/self.number_of_trees))
        np.random.RandomState(self.random_state)

        index = np.random.randint(
            0, sample_length, (self.number_of_trees, sample_length))

        return index

    def predict(self, X):
        y = []
        for tree in self.forest:
            y.append(tree.predict(X))
        mode_ = stats.mode(y, axis=0)
        mode_ = np.transpose(mode_[0])
        return mode_

    def fit(self, x_samples, y_samples):
        idx = self.bootstrap(x_samples)

        for t in range(self.number_of_trees):
            tree = tree_classifier(
                criterion="gini", max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(x_samples.iloc[idx[t]], y_samples[idx[t]])
            self.forest.append(tree)


In [15]:
# from sklearn import svm
# from cuml.svm import SVC
# import random_forest as rf


from cuml.svm import SVC
clf = cuml.ensemble.RandomForestClassifier(n_estimators=100)


# clf = SVC()

# clf = random_forest(number_of_trees=10, number_of_iteractions=1, max_depth=1)


## Sending Data to GPU

In [16]:
x = cudf.DataFrame(df.iloc[:, :-1])
x = x.astype(cupy.float32)
y = cupy.array(df.iloc[:, -1])
y = y.astype(cupy.float32)

# x = pd.DataFrame(train.iloc[:, :-1])
# y = np.array(train.iloc[:, -1])

print(x.shape)
print(y.shape)


(40000, 3681)
(40000,)


## Model Fitting

In [17]:
clf.fit(x, y)

[W] [16:50:26.917495] Using experimental backend for growing trees



RandomForestClassifier()

## Testing

In [18]:
t_df = pd.read_csv("dts/np_dataset_test.csv")
t_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 3681 entries, img_0 to allCanny
dtypes: float64(3681)
memory usage: 56.2 MB


In [19]:

# tmp = test.iloc[:, :-1].to_numpy()
tmp = np.float32(t_df.to_numpy())
y_pred = clf.predict(tmp)
print(y_pred)


[0. 0. 0. ... 1. 1. 0.]


## Export

In [20]:
expt = pd.DataFrame({"Category": y_pred})
expt = expt.astype(np.int)
expt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Category  2000 non-null   int64
dtypes: int64(1)
memory usage: 15.8 KB


/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [21]:
expt
expt.to_csv('results.csv',index=True)